In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

! pip install mlxtend

# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here: 
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [2]:
# load the data set and show the first five transaction https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv
df = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv')
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


Get the unique product that has been purchased

In [3]:
#get the unique products that has been purchased
products = df['0'].unique()
products.sort()
products

array(['Bagel', 'Bread', 'Cheese', 'Diaper', 'Eggs', 'Meat', 'Milk',
       'Pencil', 'Wine'], dtype=object)

## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [4]:
#create an itemset based on the products
itemset = set(products)
for i in range(len(df)):
    itemset.update(df.loc[i].dropna().values)
itemset = list(itemset)
print(itemset)


# encoding the feature
from mlxtend.preprocessing import TransactionEncoder

encoded_vals = []
for index, row in df.iterrows():
    labels = {}
    uncommons = list(set(itemset) - set(row))
    commons = list(set(itemset).intersection(row))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]



['Bagel', 'Wine', 'Bread', 'Eggs', 'Diaper', 'Cheese', 'Meat', 'Pencil', 'Milk']


{'Milk': 0,
 'Bagel': 0,
 'Wine': 1,
 'Bread': 1,
 'Eggs': 1,
 'Meat': 1,
 'Cheese': 1,
 'Pencil': 1,
 'Diaper': 1}

In [5]:
  # create new dataframe from the encoded features
df_encode = pd.DataFrame(encoded_vals)

  # show the new dataframe
df_encode.head()

,Milk,Bagel,Wine,Bread,Eggs,Meat,Cheese,Pencil,Diaper
0,0,0,1,1,1,1,1,1,1
1,1,0,1,1,0,1,1,1,1
2,1,0,1,0,1,1,1,0,0
3,1,0,1,0,1,1,1,0,0
4,0,0,1,0,0,1,0,1,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

In [6]:
#Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column
df_encode = df_encode.iloc[:,1:]
df_encode.head()

,Bagel,Wine,Bread,Eggs,Meat,Cheese,Pencil,Diaper
0,0,1,1,1,1,1,1,1
1,0,1,1,0,1,1,1,1
2,0,1,0,1,1,1,0,0
3,0,1,0,1,1,1,0,0
4,0,1,0,0,1,0,1,0


## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products. 
For this case study, we will min_support=0.2

In [7]:
from mlxtend.frequent_patterns import apriori

apriori(df_encode, min_support=0.2, use_colnames=True)

c:\Users\ASUS\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.425397,(Bagel)
1,0.438095,(Wine)
2,0.504762,(Bread)
3,0.438095,(Eggs)
4,0.476190,(Meat)
5,0.501587,(Cheese)
6,0.361905,(Pencil)
7,0.406349,(Diaper)
8,0.279365,"(Bread, Bagel)"
9,0.244444,"(Bread, Wine)"


Then, we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [8]:
#Then, we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6
from mlxtend.frequent_patterns import association_rules

rules = association_rules(apriori(df_encode, min_support=0.2), metric="confidence", min_threshold=0.6)
print(rules)

  antecedents consequents  antecedent support  consequent support   support  \
0         (0)         (2)            0.425397            0.504762  0.279365   
1         (1)         (5)            0.438095            0.501587  0.269841   
2         (3)         (4)            0.438095            0.476190  0.266667   
3         (3)         (5)            0.438095            0.501587  0.298413   
4         (4)         (5)            0.476190            0.501587  0.323810   
5         (5)         (4)            0.501587            0.476190  0.323810   
6      (3, 4)         (5)            0.266667            0.501587  0.215873   
7      (3, 5)         (4)            0.298413            0.476190  0.215873   
8      (4, 5)         (3)            0.323810            0.438095  0.215873   

   confidence      lift  leverage  conviction  zhangs_metric  
0    0.656716  1.301042  0.064641    1.442650       0.402687  
1    0.615942  1.227986  0.050098    1.297754       0.330409  
2    0.608696  1.278

c:\Users\ASUS\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__ and __conviction__

__antecedent__ __support__
How often items on the left (antecedent) appear in the data.

__consequent support__
How often items on the right (consequent) appear in the data.

__support__
How often both antecedent and consequent appear together.

__confidence__
Given the antecedent is present, how likely the consequent will also be present.

__lift__
How much more often the antecedent and consequent occur together than expected if they were statistically independent. Lift > 1 means they appear together more than expected, Lift < 1 means less.

__leverage__
The difference between how often the items occur together and what would be expected if they were independent. Leverage of 0 means the items are independent.

__conviction__
A measure of the rule’s implications. High conviction means the consequent strongly depends on the antecedent. Conviction is ‘inf’ (infinity) for a perfect confidence score.

For example, if 30 out of 100 transactions include Milk, the antecedent support for Milk is 0.3 or 30%. If 20 transactions include Bread, the consequent support for Bread is 0.2 or 20%. If 10 transactions include both Milk and Bread, the support for {Milk, Bread} is 0.1 or 10%. The confidence of the rule {Milk -> Bread} is then 0.1/0.3 = 0.33 or 33%, meaning that 33% of transactions that include Milk also include Bread. The lift, leverage, and conviction would require further calculations.